In [1]:
from sagas.nlu.inspectors_dataset import get_interrogative, interrogative_maps, trans_val
get_interrogative('hanem', 'hu')

def is_interr(word, interr, lang):
    return get_interrogative(word, lang)==interr
is_interr('hanem', 'but', 'hu')

True

In [2]:
from sagas.nlu.pipes import *
from sagas.nlu.inspector_pipes import *
from sagas.util.collection_util import wrap

def filter_path(*cands):
    return rx.pipe(
        ops.filter(lambda t: t.path in cands),)
def pred_interr(cand, lang):
    return rx.pipe(
        ops.filter(lambda t: is_interr(t.lemma, cand, lang)),)
def to_token():
    return rx.pipe(
        ops.map(lambda t: wrap(word=f"{t.text}/{t.lemma}",
                                     path=t.path,
                                     pos=t.upos.lower())),)

# $ nlu main_domains 'A magas tanár nem angol, hanem magyar.' hu
source=get_source('A magas tanár nem angol, hanem magyar.', 'hu')
source.pipe(
    filter_path('/conj/cc'),
    pred_interr('but', 'hu'),
    to_token(), 
).subscribe(
    on_next=lambda value: print(value),
    on_error=lambda e: print(e),
    on_completed=lambda: print('done.'),
)

{'word': 'hanem/hanem', 'path': '/conj/cc', 'pos': 'cconj'}
done.


In [43]:
pos_map={'adj': 'adjective',
         'adp': 'adposition',
         'adv': 'adverb',
         'cconj': 'conjunction',
         'sconj': 'conjunction',
        }
def get_pos(upos):    
    pos=upos.lower()
    return pos_map[pos] if pos in pos_map else pos

get_pos('adj')

'adjective'

In [48]:
pos_abbrev={'adj': 'a', 'noun':'n', 'verb':'v'}
def get_abbrev(upos):
    pos=upos.lower()
    return pos_abbrev[pos] if pos in pos_abbrev else '*'
get_abbrev('adj'), get_abbrev('conj')

('a', '*')

In [49]:
from sagas.nlu.utils import predicate, get_word_sets
kind='person'
predicate(kind, 'hungarian', 'en', get_abbrev('noun'))

True

In [45]:
from sagas.nlu.translator import translate, with_words, WordsObserver

word='magyar'
lang='hu'
pos='adj'

def trans(word, lang, pos):
    r, t = translate(word, source=lang, target='en', options={'get_pronounce'}, tracker=with_words())
    candidates=t.observer(WordsObserver).get_axis(r, get_pos(pos))
    return candidates.lower()

trans(word, lang, pos)

'hungarian'

In [52]:
source.pipe(
    filter_path('/conj'),
    ops.map(lambda t: to_obj({'word': t.text if t.upos.lower() in ['adj'] else t.lemma, **t})),
    ops.map(lambda t: to_obj({'trans': trans(t.word, lang, t.upos), **t})),
    ops.filter(lambda t: predicate(kind, t.trans, 'en', '*')),
    
    ops.map(lambda t: (t.path, t.word, t.trans, kind, t.upos.lower())),
).subscribe(
    on_next=lambda value: print(value),
    on_error=lambda e: print(e),
)

('/conj', 'magyar', 'hungarian', 'person', 'adj')


In [18]:
import json
o=wrap(x=5)
print(json.dumps(o))
print(o.x)
o2=to_obj({'y':6, **o})
print(o2.x, o2.y)
print(json.dumps(o2))

{"x": 5}
5
5 6
{"y": 6, "x": 5}


In [5]:
def xf(**kwargs):
    print(kwargs)
xf(**o)
{'y':6, **o}

{'x': 5}


{'y': 6, 'x': 5}

In [10]:
import json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
@dataclass_json
@dataclass
class token_data:
    word: str
    pos: str
    path: str

td=token_data(word='x', path='/', pos='n')
json.dumps(td.__dict__)

'{"word": "x", "pos": "n", "path": "/"}'

In [11]:
td=token_data('x', '/', 'n')